# Coherence-Gradient Chaos: Reproducibility Demo

This notebook accompanies the paper *"Chaos as a Coherence–Gradient Phenomenon: A DLSFH–SGCV–MC Analysis"*.

It illustrates two simple, **model-dependent** numerical experiments:

1. A three-node toy propagator showing coherence-gradient distance growth.
2. A VID-style pentagon loop with circulation and damping.

## 1. Environment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.cg_model import CGModel, build_pentagon_circulation_propagator
from src.example_three_node import build_three_node_propagator


## 2. Three-node toy model: distance growth $D_n$

In [ ]:
T3 = build_three_node_propagator()
model3 = CGModel(T3)
dim3 = T3.shape[0]
psi0 = np.ones(dim3)
delta = 1e-7
rng = np.random.default_rng(42)
psi0_prime = psi0 + delta * rng.normal(size=dim3)

n_steps = 200
_, _, D_n = model3.evolve_pair(psi0, psi0_prime, n_steps=n_steps)
n_all = np.arange(len(D_n))

lambda_hat, n_fit, logD_fit = model3.estimate_lambda_CG(
    psi0=psi0,
    psi0_prime=psi0_prime,
    n_steps=n_steps,
    fit_start=10,
    fit_end=150,
)
print('Three-node toy model:')
print('  spectral radius =', model3.spectral_radius())
print('  operator norm  =', model3.operator_norm())
print('  lambda_CG ≈', lambda_hat)

# Plot log D_n and fit
coeffs = np.polyfit(n_fit, logD_fit, deg=1)
logD_fit_line = np.polyval(coeffs, n_fit)

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(n_all, np.log(D_n), label='log D_n (all)')
ax.plot(n_fit, logD_fit, 'o', markersize=3, label='fit window data')
ax.plot(n_fit, logD_fit_line, '--', label='linear fit')
ax.set_xlabel('n (time step)')
ax.set_ylabel('log D_n')
ax.set_title('Three-node toy model: coherence-gradient distance growth')
ax.legend()
fig.tight_layout()
plt.show()

## 3. VID pentagon: operator norms and $\lambda_{\mathrm{CG}}$

In [ ]:
T5 = build_pentagon_circulation_propagator(eta=0.08, damping=0.02)
model5 = CGModel(T5)

dim5 = T5.shape[0]
psi0 = np.ones(dim5)
delta = 1e-7
rng = np.random.default_rng(7)
psi0_prime = psi0 + delta * rng.normal(size=dim5)

lambda_hat5, n_fit5, logD_fit5 = model5.estimate_lambda_CG(
    psi0=psi0,
    psi0_prime=psi0_prime,
    n_steps=200,
    fit_start=10,
    fit_end=150,
)
print('VID pentagon model:')
print('  spectral radius =', model5.spectral_radius())
print('  operator norm  =', model5.operator_norm())
print('  lambda_CG ≈', lambda_hat5)

for n in [1, 2, 5, 10, 20]:
    print(f'n={n:2d}, ||T_infty^n||_op =', model5.power_norm(n))